# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [5]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [6]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames (1).csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model

In [7]:
# Your code here
scaler= StandardScaler()
# Scale X_train and X_val using StandardScaler
X_train_scale = scaler.fit_transform(X_train)
X_val_scaler = scaler.transform(X_val)
# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_train_scale = pd.DataFrame(X_train_scale,columns = X_train.columns)
X_val_scale = pd.DataFrame(X_val_scaler, columns = X_val.columns)

In [8]:
# Your code here
linreg = LinearRegression()
# Create a LinearRegression model and fit it on scaled training data
linreg.fit(X_train,y_train)
# Calculate a baseline r-squared score on training data
baseline_train = linreg.score(X_train,y_train)
baseline_val = linreg.score(X_val,y_val)
print(baseline_train)
print(baseline_val)

0.7868344817421309
0.637562264303811


## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [9]:
# Your code here

# Set up data structure
interactions = []

# Find combinations of columns and loop over them
column_pair = list(combinations(X_train.columns,2))
for (col1, col2) in column_pair:
    # Make copies of X_train and X_val
    features_train = X_train.copy()
    features_val = X_val.copy()

    # Add interaction term to data
    features_train["interaction"] = features_train[col1] * features_train[col2]
    features_val["interaction"] = features_val[col1] * features_val[col2]

    # Find r-squared score (fit on training data, evaluate on validation data)
    score = LinearRegression().fit(features_train, y_train).score(features_val, y_val)

    # Append to data structure
    interactions.append(((col1, col2), score))


# Sort and subset the data structure to find the top 7
top_7_interactions = sorted(interactions, key=lambda record: record[1], reverse=True)[:7]
print("Top 7 interactions:")
print(top_7_interactions)

Top 7 interactions:
[(('LotArea', '1stFlrSF'), 0.7211105666582287), (('LotArea', 'TotalBsmtSF'), 0.7071649206211665), (('LotArea', 'GrLivArea'), 0.6690980817174615), (('LotArea', 'Fireplaces'), 0.6529699515648165), (('2ndFlrSF', 'TotRmsAbvGrd'), 0.647299489040505), (('OverallCond', 'TotalBsmtSF'), 0.6429019879233577), (('OverallQual', '2ndFlrSF'), 0.6422324294284347)]


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [10]:
# Your code here

# Loop over top 7 interactions
for record in top_7_interactions:
    # Extract column names from data structure

    # Construct new column name

    # Add new column to X_train and X_val
    col1, col2 = record[0]

    # Construct new column name
    new_col_name = col1 + "_" + col2

    # Add new column to X_train and X_val
    X_train[new_col_name] = X_train[col1] * X_train[col2]
    X_val[new_col_name] = X_val[col1] * X_val[col2]

X_train

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,LotArea_GrLivArea,LotArea_Fireplaces,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF
518,9531,6,5,794,882,914,1796,7,546,0,8406342,7567614,17117676,0,6398,3970,5484
236,8773,7,5,1414,1414,0,1414,6,494,0,12405022,12405022,12405022,0,0,7070,0
38,7922,5,7,1057,1057,0,1057,5,246,0,8373554,8373554,8373554,0,0,7399,0
1034,6305,5,7,920,954,0,954,5,240,1,6014970,5800600,6014970,6305,0,6440,0
520,10800,4,7,0,694,600,1294,7,0,0,7495200,0,13975200,0,4200,0,2400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,4426,6,5,848,848,0,848,3,420,1,3753248,3753248,3753248,4426,0,4240,0
1309,7153,6,5,1278,1294,0,1294,6,496,0,9255982,9141534,9255982,0,0,6390,0
1237,12393,7,5,847,847,1101,1948,8,434,1,10496871,10496871,24141564,12393,8808,4235,7707
214,10900,6,7,689,689,703,1392,6,299,0,7510100,7510100,15172800,0,4218,4823,4218


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)`

In [11]:
X.columns = X.columns.astype('str')

In [12]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import pandas as pd

# Ensure all column names in X_train and X_val are strings
X_train.columns = X_train.columns.astype(str)
X_val.columns = X_val.columns.astype(str)

# Set up data structure to store results
polynomials = []

# Loop over all columns in X_train
for col in X_train.columns:
    # Loop over degrees 2, 3, 4 for polynomial transformation
    for degree in (2, 3, 4):

        # Copy X_train and X_val to avoid modifying the originals
        features_train = X_train.copy()
        features_val = X_val.copy()

        # Instantiate PolynomialFeatures for the current degree
        poly = PolynomialFeatures(degree, include_bias=False)

        # Fit and transform the current column from features_train and features_val
        col_transformed_train = poly.fit_transform(features_train[[col]])
        col_transformed_val = poly.transform(features_val[[col]])

        # Convert the transformed columns to DataFrames, removing feature names by converting to numpy arrays
        col_transformed_train_df = pd.DataFrame(col_transformed_train, index=features_train.index)
        col_transformed_val_df = pd.DataFrame(col_transformed_val, index=features_val.index)

        # Drop the original column from features and add the polynomial transformed columns
        features_train = pd.concat([features_train.drop(col, axis=1), col_transformed_train_df], axis=1)
        features_val = pd.concat([features_val.drop(col, axis=1), col_transformed_val_df], axis=1)

        # Convert to NumPy arrays to remove all feature names
        features_train_np = features_train.to_numpy()
        features_val_np = features_val.to_numpy()

        # Fit a LinearRegression model and calculate R² score on validation data
        score = LinearRegression().fit(features_train_np, y_train).score(features_val_np, y_val)

        # Store the results: (column name, polynomial degree, R² score)
        polynomials.append((col, degree, score))

# Sort results by R² score in descending order and extract top 7
top_7_polynomials = sorted(polynomials, key=lambda record: record[-1], reverse=True)[:7]

# Print the top 7 results
print("Top 7 polynomials:")
for col, degree, score in top_7_polynomials:
    print(f"Column: {col}, Degree: {degree}, R² Score: {score}")


Top 7 polynomials:
Column: GrLivArea, Degree: 3, R² Score: 0.8283344538443237
Column: OverallQual, Degree: 3, R² Score: 0.806802356596642
Column: OverallQual, Degree: 4, R² Score: 0.8033455378861063
Column: OverallQual, Degree: 2, R² Score: 0.802583337313625
Column: OverallQual_2ndFlrSF, Degree: 3, R² Score: 0.7928637130010348
Column: OverallQual_2ndFlrSF, Degree: 2, R² Score: 0.7920585069426396
Column: 2ndFlrSF_TotRmsAbvGrd, Degree: 3, R² Score: 0.7804203908114113


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [13]:
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd

# Assuming top_7_polynomials contains columns with their polynomial degrees and R² values
top_polynomials = pd.DataFrame(top_7_polynomials, columns=["Column", "Degree", "R^2"])

# Drop duplicate columns based on Column name to ensure unique entries
top_polynomials.drop_duplicates(subset="Column", inplace=True)

# Loop over the remaining results
for (col, degree, _) in top_polynomials.values:

    # Handle any missing values in X_train and X_val (optional, if needed)
    X_train = X_train.fillna(X_train.mean())  # Impute missing values with mean
    X_val = X_val.fillna(X_val.mean())  # Same for validation set

    # Create polynomial terms for the specified column and degree
    poly = PolynomialFeatures(degree, include_bias=False)

    # Transform the training and validation columns using PolynomialFeatures
    train_poly = pd.DataFrame(poly.fit_transform(X_train[[col]]),
                              columns=poly.get_feature_names_out([col]),
                              index=X_train.index)  # Ensure index alignment
    val_poly = pd.DataFrame(poly.transform(X_val[[col]]),
                            columns=poly.get_feature_names_out([col]),
                            index=X_val.index)  # Ensure index alignment

    # Concatenate the new polynomial features back into the original data
    # You can drop the original column if you don't need it or keep it
    X_train = pd.concat([X_train.drop(col, axis=1), train_poly], axis=1)
    X_val = pd.concat([X_val.drop(col, axis=1), val_poly], axis=1)

# View the updated training data
X_train.head()


,LotArea,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,...,GrLivArea^3,OverallQual,OverallQual^2,OverallQual^3,OverallQual_2ndFlrSF,OverallQual_2ndFlrSF^2,OverallQual_2ndFlrSF^3,2ndFlrSF_TotRmsAbvGrd,2ndFlrSF_TotRmsAbvGrd^2,2ndFlrSF_TotRmsAbvGrd^3
518,9531,5,794,882,914,7,546,0,8406342,7567614,...,5.793206e+09,6.0,36.0,216.0,5484.0,30074256.0,1.649272e+11,6398.0,40934404.0,2.618983e+11
236,8773,5,1414,1414,0,6,494,0,12405022,12405022,...,2.827146e+09,7.0,49.0,343.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
38,7922,7,1057,1057,0,5,246,0,8373554,8373554,...,1.180932e+09,5.0,25.0,125.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
1034,6305,7,920,954,0,5,240,1,6014970,5800600,...,8.682507e+08,5.0,25.0,125.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
520,10800,7,0,694,600,7,0,0,7495200,0,...,2.166720e+09,4.0,16.0,64.0,2400.0,5760000.0,1.382400e+10,4200.0,17640000.0,7.408800e+10


Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [14]:
# Your code here

## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [15]:
# Your code here
lr = LinearRegression()
lr.fit(X_train, y_train)

print("Train R^2:", lr.score(X_train, y_train))


Train R^2: 0.8599195219296726


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [16]:
y_val = y_val.loc[X_rfe_val.index]

print(X_rfe_val.shape)  # Should now match y_val
print(y_val.shape)


NameError: name 'X_rfe_val' is not defined

In [17]:
# Your code here
for n in [5, 10, 15, 20, 25]:
    rfe = RFE(LinearRegression(), n_features_to_select=n)
    X_rfe_train = rfe.fit_transform(X_train, y_train)
    X_rfe_val = rfe.transform(X_val)

    lr = LinearRegression()
    lr.fit(X_rfe_train, y_train)

    print("Train R^2:", lr.score(X_rfe_train, y_train))
    print("Test R^2: ", lr.score(X_rfe_val, y_val))
    print(f"Using {n} out of {X_train.shape[1]} features")
    print()

Train R^2: 0.733732451385993
Test R^2:  0.696956213535687
Using 5 out of 25 features

Train R^2: 0.8136490762724309
Test R^2:  0.6526036798182819
Using 10 out of 25 features

Train R^2: 0.8216282660407169
Test R^2:  0.6572842441161743
Using 15 out of 25 features

Train R^2: 0.8368108637237922
Test R^2:  0.8497996266667296
Using 20 out of 25 features

Train R^2: 0.8599195219296726
Test R^2:  0.5862776924886013
Using 25 out of 25 features



Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [18]:
# Your code here
for alpha in [1, 10, 100, 1000, 10000]:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)

    print("Train R^2:", lasso.score(X_train, y_train))
    print("Validation R^2: ", lasso.score(X_val, y_val))
    print(f"Using { 26 - (sum(abs(lasso.coef_) < 10**(-10)))} out of {X_train.shape[1]} features")
    print("and an alpha of", alpha)
    print()

Train R^2: 0.8586021717423984
Validation R^2:  0.6811754640804095
Using 26 out of 25 features
and an alpha of 1

Train R^2: 0.8586327182578343
Validation R^2:  0.6809542873802131
Using 26 out of 25 features
and an alpha of 10

Train R^2: 0.8587665744858419
Validation R^2:  0.6792186629597178
Using 24 out of 25 features
and an alpha of 100

Train R^2: 0.8570484821351823
Validation R^2:  0.6879164973178145
Using 24 out of 25 features
and an alpha of 1000

Train R^2: 0.8554978911521149
Validation R^2:  0.6860929926230642
Using 21 out of 25 features
and an alpha of 10000



Compare the results. Which features would you choose to use?

In [ ]:
# Your written answer here

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [20]:
# Scale X_test
X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled, columns=X.columns)

# Add interactions to X_test
for record in top_7_interactions:
    col1, col2 = record[0]
    new_col_name = col1 + "_" + col2
    X_test[new_col_name] = X_test[col1] * X_test[col2]

# Add polynomials to X_val
for (col, degree, _) in top_polynomials.values:
    poly = PolynomialFeatures(degree, include_bias=False)
    poly_test = pd.DataFrame(poly.fit_transform(X_test[[col]]),
                                        columns=poly.get_feature_names_out([col]))
    X_test = pd.concat([X_test.drop(col, axis=1), poly_test], axis=1)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- 2ndFlrSF_TotRmsAbvGrd
- LotArea_1stFlrSF
- LotArea_Fireplaces
- LotArea_GrLivArea
- LotArea_TotalBsmtSF
- ...


### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [ ]:
# Your code here


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.